In [1]:
package.path = '?.lua;' .. package.path
require 'nn';
require 'rnn';
-- Copyright 2004-present Facebook. All Rights Reserved.

require 'cutorch'
require 'cunn'

local TemporalKMaxPooling, parent =
    torch.class('nn.TemporalKMaxPooling', 'nn.Module')

function TemporalKMaxPooling:__init(k, k_dynamic)
    parent.__init(self)

    self.k = k
    self.k_dynamic = k_dynamic or -1

    self.output = torch.CudaTensor()
    self.gradInput = torch.CudaTensor()
    self.indices = torch.CudaTensor()
end

function TemporalKMaxPooling:updateOutput(input)
    input = input:contiguous()
    input.nn.TemporalKMaxPooling_updateOutput(self, input)
    return self.output
end

function TemporalKMaxPooling:updateGradInput(input, gradOutput)
    input = input:contiguous()
    gradOutput = gradOutput:contiguous()

    input.nn.TemporalKMaxPooling_updateGradInput(self, input, gradOutput)
    return self.gradInput
end

In [2]:
function forward_connect(enc_rnn, dec_rnn, seq_length)
    dec_rnn.userPrevOutput = nn.rnn.recursiveCopy(dec_rnn.userPrevOutput, enc_rnn.outputs[seq_length])
    dec_rnn.userPrevCell = nn.rnn.recursiveCopy(dec_rnn.userPrevCell, enc_rnn.cells[seq_length])
end

n = 20
K = 10
W = 5
F = 30
start_tok = 3

-- Helper: convert flat index to matrix.
local function flat_to_rc(v, indices, flat_index)
   local row = math.floor((flat_index - 1) / v:size(2)) + 1
   return row, indices[row][(flat_index - 1) % v:size(2) + 1]
end

-- Helper: find kmax of vector.
local function find_k_max(pool, mat)
   local v = pool:forward(mat:t()):t()
   local orig_indices = pool.indices:t():add(1)
   return v:contiguous(), orig_indices
end

pool = nn.TemporalKMaxPooling(2*K)

result = {}
scores = torch.zeros(n+1, K):float()
hyps = torch.zeros(n+1, K, W+n+1):long()
feats = torch.zeros(n+1, K, F):float()
hyps:fill(start_tok)

i = 1
-- current beam is K by W in size
cur_beam = hyps[i]:narrow(2, i+1, W)

print('loading model... ')
--local checkpoint = torch.load('conv-model_epoch5.00_33.63.t7')
local checkpoint = torch.load('conv-model_epoch5.00_33.63.t7')
local model, model_opt = checkpoint[1], checkpoint[2]
m = {}
m.enc = model[1]
m.dec = model[2]
m.enc_rnn = model[3]
m.dec_rnn = model[4]

loading model... 	


In [3]:
-- sources = torch.CudaTensor(32, 34):long():fill(4)
source = torch.load('source.dat')
target = torch.load('target.dat')
enc_out = m.enc:forward(source)
forward_connect(m.enc_rnn, m.dec_rnn, 8)
dec_out = m.dec:forward(target)
-- input = data.make_input(article, cur_beam, cur_K)
-- model_scores = self.mlp:forward(input)


In [5]:
print(dec_out)

{
  1 : CudaTensor - size: 32x10005
  2 : CudaTensor - size: 32x10005
  3 : CudaTensor - size: 32x10005
  4 : CudaTensor - size: 32x10005
  5 : CudaTensor - size: 32x10005
  6 : CudaTensor - size: 32x10005
  7 : CudaTensor - size: 32x10005
  8 : CudaTensor - size: 32x10005
  9 : CudaTensor - size: 32x10005
  10 : CudaTensor - size: 32x10005
  11 : CudaTensor - size: 32x10005
  12 : CudaTensor - size: 32x10005
}


In [13]:
enc_out = m.enc[1]:forward(source)
forward_connect(m.enc_rnn, m.dec_rnn, 34)
--dec_out = m.dec.modules[3]:forward(target)
print(m.dec:forward(target))


[string "enc_out = m.enc[1]:forward(source)..."]:1: attempt to index a nil value
stack traceback:
	[string "enc_out = m.enc[1]:forward(source)..."]:1: in main chunk
	[C]: in function 'xpcall'
	...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:173>
	...s/candokevin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0108c1bac0: 

In [9]:
m.dec_rnn

nn.LSTM(300 -> 300)
{
  outputGate : 
    nn.Sequential {
      [input -> (1) -> (2) -> (3) -> output]
      (1): nn.ParallelTable {
        input
          |`-> (1): nn.Linear(300 -> 300)
          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
          |`-> (3): nn.CMul
           ... -> output
      }
      (2): nn.CAddTable
      (3): nn.Sigmoid
    }
    {
      _type : torch.CudaTensor
      output : CudaTensor - size: 6x300
      gradInput : 
        {
          1 : CudaTensor - size: 32x300
          2 : CudaTensor - size: 32x300
          3 : CudaTensor - size: 32x300
        }
      modules : 
        {
          1 : 
            nn.ParallelTable {
              input
                |`-> (1): nn.Linear(300 -> 300)
                |`-> (2): nn.LinearNoBias(300 -> 300) without bias
                |`-> (3): nn.CMul
                 ... -> output
            }
            {
              _type : torch.CudaTensor
              output : 
                {
                  1

    2 : 
            nn.CAddTable
            {
              gradInput : 
                {
                  1 : CudaTensor - size: 32x300
                  2 : CudaTensor - size: 32x300
                  3 : CudaTensor - size: 32x300
                }
              _type : torch.CudaTensor
              train : false
              output : CudaTensor - size: 6x300
            }
          3 : 
            nn.Sigmoid
            {
              gradInput : CudaTensor - size: 32x300
              _type : torch.CudaTensor
              train : false
              output : CudaTensor - size: 6x300
            }
        }
      train : false
    }
  step : 12
  inputSize : 300
  gradInput : CudaTensor - size: 32x300
  modules : 
    {
      1 : 
        nn.Sequential {
          [input -> (1) -> (2) -> (3) -> output]
          (1): nn.ConcatTable {
            input
              |`-> (1): nn.NarrowTable
              |`-> (2): nn.Sequential {
              |      [input -> (1) -> (2) -> 

tput
              |          |      }
              |          |      (2): nn.CMulTable
              |          |    }
              |           ... -> output
              |      }
              |      (2): nn.CAddTable
              |    }
               ... -> output
          }
          (2): nn.FlattenTable
          (3): nn.ConcatTable {
            input
              |`-> (1): nn.Sequential {
              |      [input -> (1) -> (2) -> output]
              |      (1): nn.ConcatTable {
              |        input
              |          |`-> (1): nn.Sequential {
              |          |      [input -> (1) -> (2) -> (3) -> output]
              |          |      (1): nn.ParallelTable {
              |          |        input
              |          |          |`-> (1): nn.Linear(300 -> 300)
              |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
              |          |          |`-> (3): nn.CMul
              |          |           ... -

         |      (1): nn.ParallelTable {
                    |          |          |        input
                    |          |          |          |`-> (1): nn.Linear(300 -> 300)
                    |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
                    |          |          |          |`-> (3): nn.CMul
                    |          |          |           ... -> output
                    |          |          |      }
                    |          |          |      (2): nn.CAddTable
                    |          |          |      (3): nn.Sigmoid
                    |          |          |    }
                    |          |          |`-> (2): nn.Sequential {
                    |          |          |      [input -> (1) -> (2) -> (3) -> (4) -> output]
                    |          |          |      (1): nn.NarrowTable
                    |          |          |      (2): nn.ParallelTable {
                    |          |      

e63290
                  modules : table: 0x26e63148
                  train : false
                }
            }
          train : false
        }
    }
  _gradOutputs : 
    {
      1 : CudaTensor - size: 32x300
      2 : CudaTensor - size: 32x300
      3 : CudaTensor - size: 32x300
      4 : CudaTensor - size: 32x300
      5 : CudaTensor - size: 32x300
      6 : CudaTensor - size: 32x300
      7 : CudaTensor - size: 32x300
      8 : CudaTensor - size: 32x300
      9 : CudaTensor - size: 32x300
      10 : CudaTensor - size: 32x300
      11 : CudaTensor - size: 32x300
    }
  rho : 11
  _type : torch.CudaTensor
  rmInSharedClones : true
  userPrevCell : CudaTensor - size: 6x300
  output : CudaTensor - size: 6x300
  cell : CudaTensor - size: 6x300
  zeroTensor : CudaTensor - size: 6x300
  gradCells : 
    {
      0 : CudaTensor - size: 32x300
      1 : CudaTensor - size: 32x300
      2 : CudaTensor - size: 32x300
      3 : CudaTensor - size: 32x300
      4 : CudaTensor - size: 32x30

...

x300
        }
      modules : 
        {
          1 : 
            nn.ConcatTable {
              input
                |`-> (1): nn.NarrowTable
                |`-> (2): nn.Sequential {
                |      [input -> (1) -> (2) -> output]
                |      (1): nn.ConcatTable {
                |        input
                |          |`-> (1): nn.Sequential {
                |          |      [input -> (1) -> (2) -> output]
                |          |      (1): nn.ConcatTable {
                |          |        input
                |          |          |`-> (1): nn.Sequential {
                |          |          |      [input -> (1) -> (2) -> (3) -> output]
                |          |          |      (1): nn.ParallelTable {
                |          |          |        input
                |          |          |          |`-> (1): nn.Linear(300 -> 300)
                |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
            

  3 : CudaTensor - size: 32x300
                }
              modules : 
                {
                  1 : nn.NarrowTable
                  2 : nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.Sequential {
      |      [input -> (1) -> (2) -> output]
      |      (1): nn.ConcatTable {
      |        input
      |          |`-> (1): nn.Sequential {
      |          |      [input -> (1) -> (2) -> (3) -> output]
      |          |      (1): nn.ParallelTable {
      |          |        input
      |          |          |`-> (1): nn.Linear(300 -> 300)
      |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |`-> (3): nn.CMul
      |          |           ... -> output
      |          |      }
      |          |      (2): nn.CAddTable
      |          |      (3): nn.Sigmoid
      |          |    }
      |          |`-> (2): nn.SelectTable
      |           ... -> output
      |    

-> (2) -> output]
                |          |      (1): nn.SelectTable
                |          |      (2): nn.Tanh
                |          |    }
                |           ... -> output
                |      }
                |      (2): nn.CMulTable
                |    }
                |`-> (2): nn.SelectTable
                 ... -> output
            }
            {
              _type : torch.CudaTensor
              output : 
                {
                  1 : CudaTensor - size: 6x300
                  2 : CudaTensor - size: 6x300
                }
              gradInput : 
                {
                  1 : CudaTensor - size: 32x300
                  2 : CudaTensor - size: 32x300
                  3 : CudaTensor - size: 32x300
                }
              modules : 
                {
                  1 : nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.Sequential {
      |      [input -> (1) -> (2) -

sor - size: 32x300
              _type : torch.CudaTensor
              train : false
              output : CudaTensor - size: 6x300
            }
        }
      train : false
    }
  outputSize : 300
  forgetGate : 
    nn.Sequential {
      [input -> (1) -> (2) -> (3) -> output]
      (1): nn.ParallelTable {
        input
          |`-> (1): nn.Linear(300 -> 300)
          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
          |`-> (3): nn.CMul
           ... -> output
      }
      (2): nn.CAddTable
      (3): nn.Sigmoid
    }
    {
      _type : torch.CudaTensor
      output : CudaTensor - size: 6x300
      gradInput : 
        {
          1 : CudaTensor - size: 32x300
          2 : CudaTensor - size: 32x300
          3 : CudaTensor - size: 32x300
        }
      modules : 
        {
          1 : 
            nn.ParallelTable {
              input
                |`-> (1): nn.Linear(300 -> 300)
                |`-> (2): nn.LinearNoBias(300 -> 300) without bias
            

            1 : CudaTensor - size: 32x300
                  2 : CudaTensor - size: 32x300
                  3 : CudaTensor - size: 32x300
                }
              _type : torch.CudaTensor
              train : false
              output : CudaTensor - size: 6x300
            }
          3 : 
            nn.Sigmoid
            {
              gradInput : CudaTensor - size: 32x300
              _type : torch.CudaTensor
              train : false
              output : CudaTensor - size: 6x300
            }
        }
      train : false
    }
}


In [ ]:
local final_enc_rnn = enc.modules[#enc.modules - 1]
local initial_dec_rnn = dec.modules[3]

In [10]:
m.dec

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> (5) -> (6) -> (7) -> output]
  (1): nn.LookupTable
  (2): nn.SplitTable
  (3): nn.Sequencer @ nn.LSTM(300 -> 300)
  (4): nn.Sequencer @ nn.Recursor @ nn.Dropout(0.3, busy)
  (5): nn.Sequencer @ nn.LSTM(300 -> 300)
  (6): nn.Sequencer @ nn.Recursor @ nn.Linear(300 -> 10005)
  (7): nn.Sequencer @ nn.Recursor @ nn.LogSoftMax
}
{
  _type : torch.CudaTensor
  output : 
    {
      1 : CudaTensor - size: 6x10005
      2 : CudaTensor - size: 6x10005
      3 : CudaTensor - size: 6x10005
      4 : CudaTensor - size: 6x10005
      5 : CudaTensor - size: 6x10005
      6 : CudaTensor - size: 6x10005
      7 : CudaTensor - size: 6x10005
      8 : CudaTensor - size: 6x10005
      9 : CudaTensor - size: 6x10005
      10 : CudaTensor - size: 6x10005
      11 : CudaTensor - size: 6x10005
    }
  gradInput : CudaTensor - size: 32x12
  modules : 
    {
      1 : 
        nn.LookupTable
        {
          output : CudaTensor - size: 6x11x300
       

  nn.Sequencer @ nn.LSTM(300 -> 300)
        {
          _remember : neither
          output : 
            {
              1 : CudaTensor - size: 6x300
              2 : CudaTensor - size: 6x300
              3 : CudaTensor - size: 6x300
              4 : CudaTensor - size: 6x300
              5 : CudaTensor - size: 6x300
              6 : CudaTensor - size: 6x300
              7 : CudaTensor - size: 6x300
              8 : CudaTensor - size: 6x300
              9 : CudaTensor - size: 6x300
              10 : CudaTensor - size: 6x300
              11 : CudaTensor - size: 6x300
            }
          gradInput : 
            {
              1 : CudaTensor - size: 32x300
              2 : CudaTensor - size: 32x300
              3 : CudaTensor - size: 32x300
              4 : CudaTensor - size: 32x300
              5 : CudaTensor - size: 32x300
              6 : CudaTensor - size: 32x300
              7 : CudaTensor - size: 32x300
              8 : CudaTensor - size: 32x300
           

         |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |          |`-> (3): nn.CMul
      |          |          |           ... -> output
      |          |          |      }
      |          |          |      (2): nn.CAddTable
      |          |          |      (3): nn.Sigmoid
      |          |          |    }
      |          |          |`-> (2): nn.Sequential {
      |          |          |      [input -> (1) -> (2) -> (3) -> (4) -> output]
      |          |          |      (1): nn.NarrowTable
      |          |          |      (2): nn.ParallelTable {
      |          |          |        input
      |          |          |          |`-> (1): nn.Linear(300 -> 300)
      |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |           ... -> output
      |          |          |      }
      |          |          |      (3): nn.CAddTable
      |          |          |      (4): nn.Tanh
      | 

      1 : nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.NarrowTable
      |`-> (2): nn.Sequential {
      |      [input -> (1) -> (2) -> output]
      |      (1): nn.ConcatTable {
      |        input
      |          |`-> (1): nn.Sequential {
      |          |      [input -> (1) -> (2) -> output]
      |          |      (1): nn.ConcatTable {
      |          |        input
      |          |          |`-> (1): nn.Sequential {
      |          |          |      [input -> (1) -> (2) -> (3) -> output]
      |          |          |      (1): nn.ParallelTable {
      |          |          |        input
      |          |          |          |`-> (1): nn.Linear(300 -> 300)
      |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |          |`-> (3): nn.CMul
      |          |          |           ... -> output
      |          |          |      }
      |          | 

): nn.CMulTable
      |    }
      |`-> (2): nn.SelectTable
       ... -> output
  }
}
                  2 : nn.Sequential {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.NarrowTable
      |`-> (2): nn.Sequential {
      |      [input -> (1) -> (2) -> output]
      |      (1): nn.ConcatTable {
      |        input
      |          |`-> (1): nn.Sequential {
      |          |      [input -> (1) -> (2) -> output]
      |          |      (1): nn.ConcatTable {
      |          |        input
      |          |          |`-> (1): nn.Sequential {
      |          |          |      [input -> (1) -> (2) -> (3) -> output]
      |          |          |      (1): nn.ParallelTable {
      |          |          |        input
      |          |          |          |`-> (1): nn.Linear(300 -> 300)
      |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |          |`-> (3): nn.CMul
      |    

...

tial {
  [input -> (1) -> (2) -> (3) -> output]
  (1): nn.ParallelTable {
    input
      |`-> (1): nn.Linear(300 -> 300)
      |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |`-> (3): nn.CMul
       ... -> output
  }
  (2): nn.CAddTable
  (3): nn.Sigmoid
}
                  step : 12
                  output : CudaTensor - size: 6x300
                  gradInput : CudaTensor - size: 32x300
                  modules : table: 0x26f1b408
                  _gradOutputs : table: 0x27333d40
                  rho : 11
                  nSharedClone : 12
                  rmInSharedClones : true
                  zeroTensor : CudaTensor - size: 6x300
                  sharedClones : table: 0x27371200
                  outputs : table: 0x27206bb0
                  cellLayer : nn.Sequential {
  [input -> (1) -> (2) -> output]
  (1): nn.ConcatTable {
    input
      |`-> (1): nn.Sequential {
      |      [input -> (1) -> (2) -> output]
      |      (1): nn.ConcatTable {
      |        

    |      (1): nn.ParallelTable {
      |          |          |        input
      |          |          |          |`-> (1): nn.Linear(300 -> 300)
      |          |          |          |`-> (2): nn.LinearNoBias(300 -> 300) without bias
      |          |          |          |`-> (3): nn.CMul
      |          |          |           ... -> output
      |          |          |      }
      |          |          |      (2): nn.CAddTable
      |          |          |      (3): nn.Sigmoid
      |          |          |    }
      |          |          |`-> (2): nn.SelectTable
      |          |           ... -> output
      |          |      }
      |          |      (2): nn.CMulTable
      |          |    }
      |          |`-> (2): nn.Sequential {
      |          |      [input -> (1) -> (2) -> output]
      |          |      (1): nn.ConcatTable {
      |          |        input
      |          |          |`-> (1): nn.Sequential {
      |          |          |      [input -> (1) -> (2)


}
                  hiddenLayer : nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> output]
  (1): nn.NarrowTable
  (2): nn.ParallelTable {
    input
      |`-> (1): nn.Linear(300 -> 300)
      |`-> (2): nn.LinearNoBias(300 -> 300) without bias
       ... -> output
  }
  (3): nn.CAddTable
  (4): nn.Tanh
}
                }
            }
          dpnn_mediumEmpty : 
            {
              1 : output
              2 : gradInput
              3 : momGradParams
              4 : dpnn_input
              5 : _output
            }
        }
      6 : 
        nn.Sequencer @ nn.Recursor @ nn.Linear(300 -> 10005)
        {
          _remember : neither
          output : 
            {
              1 : CudaTensor - size: 6x10005
              2 : CudaTensor - size: 6x10005
              3 : CudaTensor - size: 6x10005
              4 : CudaTensor - size: 6x10005
              5 : CudaTensor - size: 6x10005
              6 : CudaTensor - size: 6x10005
              7 : CudaTensor -

-> 10005)
                  10 : nn.Linear(300 -> 10005)
                  11 : nn.Linear(300 -> 10005)
                  12 : nn.Linear(300 -> 10005)
                }
              recurrentModule : 
                nn.Linear(300 -> 10005)
                {
                  gradBias : CudaTensor - size: 10005
                  output : CudaTensor - size: 6x10005
                  gradInput : CudaTensor - size: 32x300
                  weight : CudaTensor - size: 10005x300
                  _type : torch.CudaTensor
                  gradWeight : CudaTensor - size: 10005x300
                  addBuffer : CudaTensor - size: 6
                  train : false
                  bias : CudaTensor - size: 10005
                }
            }
          _type : torch.CudaTensor
          modules : 
            {
              1 : 
                nn.Recursor @ nn.Linear(300 -> 10005)
                {
                  step : 12
                  outputs : table: 0x19ba9370
                 

              }
              _gradOutputs : table: 0x19855838
              rho : 11
              train : false
              _type : torch.CudaTensor
              rmInSharedClones : true
              module : 
                nn.LogSoftMax
                {
                  gradInput : CudaTensor - size: 32x10005
                  _type : torch.CudaTensor
                  train : false
                  output : CudaTensor - size: 6x10005
                }
              nSharedClone : 12
              sharedClones : 
                {
                  1 : nn.LogSoftMax
                  2 : nn.LogSoftMax
                  3 : nn.LogSoftMax
                  4 : nn.LogSoftMax
                  5 : nn.LogSoftMax
                  6 : nn.LogSoftMax
                  7 : nn.LogSoftMax
                  8 : nn.LogSoftMax
                  9 : nn.LogSoftMax
                  10 : nn.LogSoftMax
                  11 : nn.LogSoftMax
                  12 : nn.LogSoftMax
                }

In [16]:
test_str = torch.serialize(mlp)

string length overflow
stack traceback:
	[C]: in function 'serialize'
	[string "test_str = torch.serialize(mlp)..."]:1: in main chunk
	[C]: in function 'xpcall'
	...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:209: in function <...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:173>
	...s/candokevin/torch/install/share/lua/5.1/lzmq/poller.lua:75: in function 'poll'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:307: in function 'poll'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:325: in function 'sleep_ex'
	...andokevin/torch/install/share/lua/5.1/lzmq/impl/loop.lua:370: in function 'start'
	...s/candokevin/torch/install/share/lua/5.1/itorch/main.lua:381: in main chunk
	[C]: in function 'require'
	(command line):1: in main chunk
	[C]: at 0x0106f87ac0: 